# DTOcean Mooring and Foundations Example

Note, this example assumes the DTOcean database and the Electrical Sub-Systems Module has been installed

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [ ]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu
from dtocean_core.pipeline import Tree

In [ ]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [ ]:
# Bring up the logger
start_logging()

## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [ ]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
pipe_tree = Tree()

## Create a new project

In [ ]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

## Set the device type

In [ ]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type" 
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Tidal Floating")
my_var.read(new_core, new_project)

## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [ ]:
project_menu.initiate_pipeline(new_core, new_project)

## Discover available modules

In [ ]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [ ]:
module_name = 'Mooring and Foundations'
module_menu.activate(new_core, new_project, module_name)

## Check the status of the module inputs

In [ ]:
mooring_branch = pipe_tree.get_branch(new_core, new_project, 'Mooring and Foundations')
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [ ]:
project_menu.initiate_dataflow(new_core, new_project)

## Load test data

Prepare the test data for loading. The test_data directory of the source code should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [ ]:
%run test_data/inputs_wp4.py

In [ ]:
mooring_branch.read_test_data(new_core,
                              new_project,
                              "test_data/inputs_wp4.pkl")

## Check if the module can be executed

In [ ]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

In [ ]:
input_status = mooring_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [ ]:
module_menu.execute_current(new_core, new_project)

## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [ ]:
output_status = mooring_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [ ]:
economics = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_foundations_economics_data")
economics.get_value(new_core, new_project)

In [ ]:
moorings = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_component_data")
moorings.get_value(new_core, new_project)

In [ ]:
lines = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_line_data")
lines.get_value(new_core, new_project)

In [ ]:
umbilicals = mooring_branch.get_output_variable(new_core, new_project, "project.umbilical_cable_data")
umbilicals.get_value(new_core, new_project)

In [ ]:
dimensions = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_dimensions")
dimensions.get_value(new_core, new_project)

In [ ]:
network = mooring_branch.get_output_variable(new_core, new_project, "project.moorings_foundations_network")
net_val = network.get_value(new_core, new_project)

In [ ]:
import pprint
pprint.pprint(net_val)